<a href="https://colab.research.google.com/github/jphacks/D_2201/blob/master/preprocess/make_all_pol_share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 枠の用意

In [1]:
!wget https://github.com/JLLT-Japan/jphacks2022/raw/main/data/official_land_prices_all.zip
!unzip /content/official_land_prices_all.zip

--2022-11-05 21:13:25--  https://github.com/JLLT-Japan/jphacks2022/raw/main/data/official_land_prices_all.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JLLT-Japan/jphacks2022/main/data/official_land_prices_all.zip [following]
--2022-11-05 21:13:25--  https://raw.githubusercontent.com/JLLT-Japan/jphacks2022/main/data/official_land_prices_all.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7607062 (7.3M) [application/zip]
Saving to: ‘official_land_prices_all.zip’

official_land_price 100%[===================>]   7.25M  --.-KB/s    in 0.09s   

2022-11-05 21:13:25 (78.6 MB/s) - ‘official_

In [2]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', 5)

In [3]:
df_jll = pd.read_csv("/content/official_land_prices_all.csv")
df_jll.describe()

,longitude,latitude,...,seireki,price
count,1039720.00,1039720.00,...,1039720.00,1039720.00
mean,137.22,35.70,...,2002.50,139375.83
std,3.31,2.36,...,11.54,738839.60
min,124.15,24.34,...,1983.00,0.00
25%,135.43,34.69,...,1992.75,0.00
50%,137.98,35.39,...,2002.50,31800.00
75%,139.73,35.95,...,2012.25,113000.00
max,145.60,45.42,...,2022.00,57700000.00


In [4]:
df_jll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039720 entries, 0 to 1039719
Data columns (total 34 columns):
 #   Column                                Non-Null Count    Dtype  
---  ------                                --------------    -----  
 0   longitude                             1039720 non-null  float64
 1   latitude                              1039720 non-null  float64
 2   sequential_number                     1039720 non-null  int64  
 3   district                              1039720 non-null  object 
 4   address                               1039720 non-null  object 
 5   street_address                        1039720 non-null  object 
 6   lot_size                              1039720 non-null  int64  
 7   indication_use                        1039720 non-null  object 
 8   building_structure                    1039720 non-null  object 
 9   shape_category                        1039720 non-null  object 
 10  width_ratio                           1039720 non-null

In [5]:
poi = [34.4,34.8,135.4,135.8]

In [6]:
def lat2id(lat):
  #min-lat = 24.34
  id = (lat - 24)//0.4
  return int(id)
def lon2id(lon):
  #min-lon = 124.5
  id = (lon - 123.8)//0.4
  return int(id)
print(lat2id(24.34))
print(lat2id(45.42))
print(lon2id(124.15))
print(lon2id(145.60))

0
53
0
54


In [7]:
def id2lat(id):
  mlat = id*0.4+24
  return round(mlat,2)
print(id2lat(53))
def id2lon(id):
  mlon = id*0.4+123.8
  return round(mlon,2)
print(id2lon(54))

45.2
145.4


In [8]:
import numpy as np
from tqdm.notebook import tqdm

In [9]:
data_lat = list(df_jll["latitude"])
data_lon = list(df_jll["longitude"])
data_area=list(df_jll["current_status_land_surrounding_area"])
data_count = len(list(df_jll["latitude"]))
print(data_count)

1039720


In [10]:
pols = np.zeros((54,55)) #lat,lon
for i in tqdm(range(data_count)):
  idlat = lat2id(data_lat[i])
  idlon = lon2id(data_lon[i])
  pols[idlat][idlon] +=1

  0%|          | 0/1039720 [00:00<?, ?it/s]

In [11]:
usepols = []
for idlat in range(54):
  for idlon in range(55):
    if pols[idlat][idlon] != 0:
      usepols.append([id2lat(idlat),id2lon(idlon)])
print(len(usepols),54*55)

333 2970


In [12]:
from collections import defaultdict

level1 = defaultdict(dict) #0.4
level1_keys = []
level2 = defaultdict(dict) #0.2
level2_keys = []
level3 = defaultdict(dict) #0.04
level3_keys = []
level4 = defaultdict(dict) #0.01
level4_keys = []
for pol in tqdm(usepols):
  #level1 per 0.4deg
  level1[pol[0]][pol[1]] = {"score": 0, "ascore":0,"bscore":0,"cscore":0,"sscore":0,}
  level1_keys.append([pol[0],pol[1]])
  #level2 per 0.2deg
  for lat in range(2):
    for lon in range(2):
      level2[round(pol[0]+(0.2*lat),2)][round(pol[1]+(0.2)*lon,2)] = {"score": 0, "ascore":0,"bscore":0,"cscore":0,"sscore":0,}
      level2_keys.append([round(pol[0]+(0.2*lat),2),round(pol[1]+(0.2*lon),2)])
  #level3 per 0.04deg
  for lat in range(10):
    for lon in range(10):
      level3[round(pol[0]+(0.04*lat),2)][round(pol[1]+(0.04)*lon,2)] = {"score": 0, "ascore":0,"bscore":0,"cscore":0,"sscore":0,}
      level3_keys.append([round(pol[0]+(0.04*lat),2),round(pol[1]+(0.04*lon),2)])
  #level4 per 0.01deg
  for lat in range(40):
    for lon in range(40):
      level4[round(pol[0]+(0.01*lat),2)][round(pol[1]+(0.01*lon),2)] = {"score": 0, "ascore":0,"bscore":0,"cscore":0,"sscore":0,}
      level4_keys.append([round(pol[0]+(0.01*lat),2), round(pol[1]+(0.01*lon),2)])
print(len(level1_keys))
print(len(level2_keys))
print(len(level3_keys))
print(len(level4_keys))

  0%|          | 0/333 [00:00<?, ?it/s]

333
1332
33300
532800


In [13]:
# それぞれのLevelにてlatをその区画の最小lat(鍵)に変換する
def lat2l1(geo):
  ret = ((geo*100-2400)//40) * 40 + 2400
  return ret/100
def lat2l2(geo):
  ret = ((geo*100-2400)//20) * 20 + 2400
  return ret/100
def lat2l3(geo):
  ret = ((geo*100-2400)//4) * 4 + 2400
  return ret/100
def lat2l4(geo):
  ret = ((geo*100-2400)//1) * 1 + 2400
  return ret/100
# それぞれのLevelにてlonをその区画の最小lon(鍵)に変換する
def lon2l1(geo):
  ret = ((geo*100-12380)//40) * 40 + 12380
  return ret/100
def lon2l2(geo):
  ret = ((geo*100-12380)//20) * 20 + 12380
  return ret/100
def lon2l3(geo):
  ret = int((geo*100-12380)//4) * 4 + 12380
  return ret/100
def lon2l4(geo):
  ret = ((geo*100-12380)//1) * 1 + 12380
  return ret/100

## bscore(building scoreの処理)

### Phase1 各JLLデータを各枠に割り振り

In [14]:
def area2prescore(building):
    #current_status_land_surrounding_areaを数値化
    if '小中規模' in  building :
        return 3 
    if '中層' in  building :
        return 5   
    elif '中高層' in building :
        return 5
    elif '中規模' in building :
        return 5
    elif '駅前商業地域' in building :
        return 5
    elif '高層' in building :
        return 10
    elif '大規模' in  building :
        return 10
    else: 
        return 1

In [15]:
data_lat = list(df_jll["latitude"])
data_lon = list(df_jll["longitude"])
data_area=list(df_jll["current_status_land_surrounding_area"])
data_count = len(list(df_jll["latitude"]))
print(data_count)

1039720


In [16]:
for i in tqdm(range(data_count)):
  lat = data_lat[i]
  lon = data_lon[i]
  prescore = area2prescore(data_area[i])
  level1[lat2l1(lat)][lon2l1(lon)]["bscore"] +=prescore
  level2[lat2l2(lat)][lon2l2(lon)]["bscore"] +=prescore
  level3[lat2l3(lat)][lon2l3(lon)]["bscore"] +=prescore
  level4[lat2l4(lat)][lon2l4(lon)]["bscore"] +=prescore

  0%|          | 0/1039720 [00:00<?, ?it/s]

### Phase2 集計とbscore確定

In [17]:
def prescore2bscore(prescore):
  PARAM = 150
  if(prescore>=PARAM):
      bu_score = 0
  else:
      bu_score = 1 - (prescore / PARAM)
  return round(bu_score, 2)

In [18]:
for key in tqdm(level1_keys):
  level1[key[0]][key[1]]["bscore"] = prescore2bscore(level1[key[0]][key[1]]["bscore"])
for key in tqdm(level2_keys):
  level2[key[0]][key[1]]["bscore"] = prescore2bscore(level2[key[0]][key[1]]["bscore"])
for key in tqdm(level3_keys):
  level3[key[0]][key[1]]["bscore"] = prescore2bscore(level3[key[0]][key[1]]["bscore"])
for key in tqdm(level4_keys):
  level4[key[0]][key[1]]["bscore"] = prescore2bscore(level4[key[0]][key[1]]["bscore"])

  0%|          | 0/333 [00:00<?, ?it/s]

  0%|          | 0/1332 [00:00<?, ?it/s]

  0%|          | 0/33300 [00:00<?, ?it/s]

  0%|          | 0/532800 [00:00<?, ?it/s]

## 統合(score算出と書き出し)

In [19]:
for key in tqdm(level1_keys):
  level1[key[0]][key[1]]["score"] = level1[key[0]][key[1]]["bscore"] * 100
for key in tqdm(level2_keys):
  level2[key[0]][key[1]]["score"] = level2[key[0]][key[1]]["bscore"] * 100
for key in tqdm(level3_keys):
  level3[key[0]][key[1]]["score"] = level3[key[0]][key[1]]["bscore"] * 100
for key in tqdm(level4_keys):
  level4[key[0]][key[1]]["score"] = level4[key[0]][key[1]]["bscore"] * 100

  0%|          | 0/333 [00:00<?, ?it/s]

  0%|          | 0/1332 [00:00<?, ?it/s]

  0%|          | 0/33300 [00:00<?, ?it/s]

  0%|          | 0/532800 [00:00<?, ?it/s]

In [20]:
!pip install -q geojson

In [21]:

from geojson import Polygon, Feature, FeatureCollection, dump

In [22]:
outfile = "building_all"
# Level1
ft_level1 = []
for key in tqdm(level1_keys):
  lat_min, lon_min = key
  lat_max = lat_min + 0.4
  lon_max = lon_min + 0.4
  geopoly = Polygon([[(lon_min, lat_min), (lon_min, lat_max), (lon_max, lat_max), (lon_max, lat_min)]])
  ft = Feature(geometry = geopoly, properties = {'score': level1[key[0]][key[1]]["score"]})
  ft_level1.append(ft)
ft_colct = FeatureCollection(ft_level1)
with open(outfile + "_level1.geojson", 'w') as f:
    dump(ft_colct, f, separators=(',', ':'))

#Level2
ft_level2 = []
for key in tqdm(level2_keys):
  lat_min, lon_min = key
  lat_max = lat_min + 0.2
  lon_max = lon_min + 0.2
  geopoly = Polygon([[(lon_min, lat_min), (lon_min, lat_max), (lon_max, lat_max), (lon_max, lat_min)]])
  ft = Feature(geometry = geopoly, properties = {'score': level2[key[0]][key[1]]["score"]})
  ft_level2.append(ft)
ft_colct = FeatureCollection(ft_level2)
with open(outfile + "_level2.geojson", 'w') as f:
    dump(ft_colct, f, separators=(',', ':'))

#Level3
ft_level3 = []
for key in tqdm(level3_keys):
  lat_min, lon_min = key
  lat_max = lat_min + 0.04
  lon_max = lon_min + 0.04
  geopoly = Polygon([[(lon_min, lat_min), (lon_min, lat_max), (lon_max, lat_max), (lon_max, lat_min)]])
  ft = Feature(geometry = geopoly, properties = {'score': level3[key[0]][key[1]]["score"]})
  ft_level3.append(ft)
ft_colct = FeatureCollection(ft_level3)
with open(outfile + "_level3.geojson", 'w') as f:
    dump(ft_colct, f, separators=(',', ':'))

#Level4
ft_level4 = []
for key in tqdm(level4_keys):
  lat_min, lon_min = key
  lat_max = lat_min + 0.01
  lon_max = lon_min + 0.01
  geopoly = Polygon([[(lon_min, lat_min), (lon_min, lat_max), (lon_max, lat_max), (lon_max, lat_min)]])
  ft = Feature(geometry = geopoly, properties = {'score': level4[key[0]][key[1]]["score"]})
  ft_level4.append(ft)
ft_colct = FeatureCollection(ft_level4)
with open(outfile + "_level4.geojson", 'w') as f:
    dump(ft_colct, f, separators=(',', ':'))
print("すべての書き出しが完了しました")

  0%|          | 0/333 [00:00<?, ?it/s]

  0%|          | 0/1332 [00:00<?, ?it/s]

  0%|          | 0/33300 [00:00<?, ?it/s]

  0%|          | 0/532800 [00:00<?, ?it/s]

すべての書き出しが完了しました
